In [16]:
from pyspark.sql import functions as F

StatementMeta(, ffc5e1ad-0ab6-4031-8ccc-ec09300bb51e, 84, Finished, Available, Finished)

In [17]:
Full_Load_Data_path = "Files/Bronze/Full_Load_Data"
Incremental_Load_Data_path = "Files/Bronze/Incremental_Load_Data"
Silver_layer_path = "Files/Silver/Final_Sales_Data"

StatementMeta(, ffc5e1ad-0ab6-4031-8ccc-ec09300bb51e, 85, Finished, Available, Finished)

In [4]:
# Read Full Data from Bronze Layer

bronze_full_load_df = spark.read.parquet(Full_Load_Data_path)
display(bronze_full_load_df)

StatementMeta(, ffc5e1ad-0ab6-4031-8ccc-ec09300bb51e, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0801197e-aae9-4d4f-8460-4b9312799673)

In [5]:
# Read Incremental Data from Bronze Layer

bronze_incremental_load_df = spark.read.parquet(Incremental_Load_Data_path)
display(bronze_incremental_load_df)

StatementMeta(, ffc5e1ad-0ab6-4031-8ccc-ec09300bb51e, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e59b52c9-57a4-462c-98bf-0b8520b544e8)

In [7]:
# Combine Full Load and Incremental Load DataFrames using union

merged_data_df = bronze_full_load_df.union(bronze_incremental_load_df)
display(merged_data_df)
merged_data_df.count()

StatementMeta(, ffc5e1ad-0ab6-4031-8ccc-ec09300bb51e, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 273a3b1a-054f-4017-9c92-a1df2d0a6c88)

110000

In [11]:
merged_data_df = merged_data_df.withColumn(
    "GROSS_AMOUNT", 
    F.col("QUANTITY_SOLD") * F.col("UNIT_PRICE")
).withColumn(
    "PROFIT_MARGIN", 
    (F.col("TOTAL_AMOUNT") - (F.col("QUANTITY_SOLD") * F.col("UNIT_PRICE") * 0.7)) / F.col("TOTAL_AMOUNT")
).withColumn(
    "SALES_YEAR", 
    F.year(F.col("SALES_DATE"))
)

display(merged_data_df)

StatementMeta(, ffc5e1ad-0ab6-4031-8ccc-ec09300bb51e, 79, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 70005690-f2d6-4b3e-9e02-cdfee4c1225c)

In [19]:
# Appending incremental data to the existing Silver Layer Delta table

merged_data_df.write.format("delta").mode("append").save(Silver_layer_path)

print(f"Data has been successfully written to Silver Layer in Delta format at {Silver_layer_path}")

StatementMeta(, ffc5e1ad-0ab6-4031-8ccc-ec09300bb51e, 87, Finished, Available, Finished)

Data has been successfully written to Silver Layer in Delta format at Files/Silver/Final_Sales_Data
